In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('./data/demo_03.xlsx', sheet_name='Sheet1')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9959 entries, 0 to 9958
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   订单编号    9959 non-null   object 
 1   产品名称    9959 non-null   object 
 2   一级分类    9959 non-null   object 
 3   二级分类    9959 non-null   object 
 4   省份      9959 non-null   object 
 5   大区      9959 non-null   object 
 6   数量      9959 non-null   int64  
 7   金额      9959 non-null   float64
 8   利润      9959 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 700.4+ KB


# groupby分组函数
- groupby函数 + 聚合函数

## 一列分组一列聚合

In [10]:
df.groupby('二级分类')['数量'].sum().reset_index()
# df.groupby('二级分类')['数量'].sum().to_frame()

,二级分类,数量
0,书架,2401
1,信封,2281
2,器具,2134
3,复印机,2139
4,收纳具,2918
5,标签,2106
6,桌子,620
7,椅子,3172
8,用具,2298
9,用品,2266


## 一列分组多列聚合

In [13]:
df.groupby('一级分类')[['数量', '金额']].sum().reset_index()

,一级分类,数量,金额
0,办公,21401,4865589.87
1,家具,8491,5734340.95
2,技术,7642,5469023.47


## 多列分组多列聚合

In [14]:
df.groupby(['一级分类', '二级分类'])[['数量', '金额']].sum().reset_index()

,一级分类,二级分类,数量,金额
0,办公,信封,2281,287486.08
1,办公,器具,2134,2160182.99
2,办公,收纳具,2918,1152527.74
3,办公,标签,2106,97077.96
4,办公,用品,2266,287970.51
5,办公,系固件,2272,129010.69
6,办公,纸张,2063,263334.12
7,办公,美术,2009,196222.82
8,办公,装订机,3352,291776.96
9,家具,书架,2401,2307203.20


# 对分组数据进行遍历

In [16]:
groups = df.groupby('一级分类')
for key, tb in groups:
    print(key)
    print(tb)
    print('-'*100)

办公
                 订单编号             产品名称 一级分类 二级分类  省份  大区  数量       金额       利润
0     US-2021-1357144  办公用-用品-10002717   办公   用品  浙江  华东   2   129.70   -60.70
1     CN-2021-1973789  办公用-信封-10004832   办公   信封  四川  西南   2   125.44    42.56
2     CN-2021-1973789  办公用-装订-10001505   办公  装订机  四川  西南   2    31.92     4.20
3     US-2021-3017568  办公用-用品-10003746   办公   用品  江苏  华东   4   321.22   -27.10
4     CN-2020-2975416  办公用-器具-10003452   办公   器具  广东  中南   3  1375.92   550.20
...               ...              ...  ...  ...  ..  ..  ..      ...      ...
9948  CN-2018-1902956  办公用-装订-10002129   办公  装订机  江苏  华东   2   296.35   -93.97
9949  CN-2018-1902956  办公用-装订-10000994   办公  装订机  江苏  华东   3   120.96   -12.18
9951  CN-2019-3665640  办公用-收纳-10002744   办公  收纳具  湖北  中南   4   192.08    22.96
9955  CN-2021-4318875  办公用-收纳-10003383   办公  收纳具  浙江  华东   2   447.72    35.56
9958  CN-2018-3557528  办公用-器具-10002023   办公   器具  浙江  华东   5  7244.58 -3501.82

[5687 rows x 9 columns]
------------------------

# groupby+agg函数

## 指定列使用多个聚合函数

In [17]:
df.groupby('一级分类')[['数量', '金额']].agg(['mean', 'sum'])

数量                  金额            
          mean    sum         mean         sum
一级分类                                          
办公    3.763144  21401   855.563543  4865589.87
家具    3.783868   8491  2555.410406  5734340.95
技术    3.768245   7642  2696.757135  5469023.47

## 指定列使用指定聚合函数

In [4]:
df.groupby('一级分类').agg({'数量': ['sum','mean'], '金额': 'mean', '利润': 'sum'}).reset_index()

一级分类     数量                     金额         利润
          sum      mean         mean        sum
0   办公  21401  3.763144   855.563543  757640.43
1   家具   8491  3.783868  2555.410406  638735.62
2   技术   7642  3.768245  2696.757135  751162.91

# 自定义函数分组统计

- 统计频数最高（行计数最多）的二级分类
- 以及总数量和总金额

In [36]:
# df['二级分类'].value_counts().index[0] # 频数最高的二级分类
# df['二级分类'].value_counts().iloc[0] # 频数最高的值
df.agg({
    '二级分类': lambda x: x.value_counts().index[0],
    '省份': lambda x: x.value_counts().index[0],
    '数量': 'sum',
    '金额': 'sum'}).reset_index()

,index,0
0,二级分类,装订机
1,省份,山东
2,数量,37534
3,金额,16068954.29


- 以一级分类为分组，统计销售数量最多的二级分类

In [41]:
df['一级分类'].unique()

array(['办公', '技术', '家具'], dtype=object)

In [45]:
df[df['一级分类'] == '办公'].groupby('二级分类')['数量'].sum().idxmax()

'装订机'

In [46]:
df[df['一级分类'] == '技术'].groupby('二级分类')['数量'].sum().idxmax()

'电话'

In [47]:
df[df['一级分类'] == '家具'].groupby('二级分类')['数量'].sum().idxmax()

'椅子'

In [49]:
grouped = df[['一级分类', '二级分类', '数量']].groupby('一级分类')
grouped.apply(lambda x: x.groupby('二级分类')['数量'].sum().idxmax()).reset_index()

,一级分类,0
0,办公,装订机
1,家具,椅子
2,技术,电话


# groupby+transform函数
- pd.DataFrame.transform()
- 统计各省的金额以及占所属大区的比例

In [9]:
tb = df.groupby(['大区', '省份']).agg({'金额': np.sum}).reset_index()

In [10]:
tb['大区金额'] = tb.groupby('大区')['金额'].transform('sum')

In [11]:
tb['金额占比'] = tb['金额'] / tb['大区金额']

In [12]:
tb['金额占比'] = tb['金额占比'].apply(lambda x: '{:.2%}'.format(x))

In [13]:
tb

,大区,省份,金额,大区金额,金额占比
0,东北,吉林,640196.59,2681567.51,23.87%
1,东北,辽宁,862569.72,2681567.51,32.17%
2,东北,黑龙江,1178801.20,2681567.51,43.96%
3,中南,广东,1452929.53,4137415.10,35.12%
4,中南,广西,377653.82,4137415.10,9.13%
5,中南,河南,853574.77,4137415.10,20.63%
6,中南,海南,107854.42,4137415.10,2.61%
7,中南,湖北,621960.36,4137415.10,15.03%
8,中南,湖南,723442.20,4137415.10,17.49%
9,华东,上海,582450.57,4684506.52,12.43%


# 多列合并计算

## groupby+字典+聚合函数

In [7]:
df = pd.read_excel('./data/demo_03.xlsx', sheet_name='Sheet2')
df.head()

,二级分类,北京销量,上海销量,广州销量,深圳销量,长沙销量
0,用品,1224,1717,2372,877,1977
1,信封,779,1952,1065,1162,1363
2,装订机,2765,731,2184,1101,1149
3,器具,343,1266,1255,1290,594
4,设备,614,253,1414,1352,640


In [13]:
# df.columns
dct = {
    '北京销量': '北上广深',
    '上海销量': '北上广深',
    '广州销量': '北上广深',
    '深圳销量': '北上广深',
    '长沙销量': '长沙'}
tb = df.set_index('二级分类')
tb.groupby(dct, axis=1).sum().reset_index()

,二级分类,北上广深,长沙
0,用品,6190,1977
1,信封,4958,1363
2,装订机,6781,1149
3,器具,4154,594
4,设备,3633,640
5,椅子,5660,704
6,纸张,5364,1680
7,系固件,4377,512
8,复印机,1363,439
9,配件,1627,408


In [42]:
tb = df.set_index('二级分类').T
tb.index = tb.index.map(dct)
tb.groupby(tb.index).sum().T

,北上广深,长沙
二级分类,,
用品,6190,1977
信封,4958,1363
装订机,6781,1149
器具,4154,594
设备,3633,640
椅子,5660,704
纸张,5364,1680
系固件,4377,512
复印机,1363,439


## groupby+Series+聚合函数

In [43]:
df = pd.read_excel('./data/demo_03.xlsx', sheet_name='Sheet2')

In [49]:
# dct.keys()
# dct.values()
tb = df.set_index('二级分类')
s = pd.Series(index=['北京销量', '上海销量', '广州销量', '深圳销量', '长沙销量'],
              data=['北上广深', '北上广深', '北上广深', '北上广深', '长沙'])
tb.groupby(s, axis=1).sum()

,北上广深,长沙
二级分类,,
用品,6190,1977
信封,4958,1363
装订机,6781,1149
器具,4154,594
设备,3633,640
椅子,5660,704
纸张,5364,1680
系固件,4377,512
复印机,1363,439


# pivot_table透视表

In [51]:
df = pd.read_excel('./data/demo_03.xlsx', sheet_name='Sheet1')
df.head()

,订单编号,产品名称,一级分类,二级分类,省份,大区,数量,金额,利润
0,US-2021-1357144,办公用-用品-10002717,办公,用品,浙江,华东,2,129.70,-60.70
1,CN-2021-1973789,办公用-信封-10004832,办公,信封,四川,西南,2,125.44,42.56
2,CN-2021-1973789,办公用-装订-10001505,办公,装订机,四川,西南,2,31.92,4.20
3,US-2021-3017568,办公用-用品-10003746,办公,用品,江苏,华东,4,321.22,-27.10
4,CN-2020-2975416,办公用-器具-10003452,办公,器具,广东,中南,3,1375.92,550.20


In [55]:
# data, df数据源
# values, Excel数据透视表的值
# index, Excel数据透视表的行
# columns, Excel数据透视表的列
# aggfunc, Excel数据透视表的计算类型(汇总依据)
# fill_value, 填充空值
# margins, 是否显示合计
# dropna, 是否删除缺失行（整行缺失）
# margins_name, 合计的别名
pd.pivot_table(
    data=df,
    index='大区',
    columns='一级分类',
    values='金额',
    aggfunc='sum',
    margins=True,
    margins_name='总计',
    fill_value=0)

一级分类,办公,家具,技术,总计
大区,,,,
东北,824673.09,920698.44,936195.98,2681567.51
中南,1270911.26,1399928.21,1466575.63,4137415.10
华东,1408628.63,1676224.16,1599653.73,4684506.52
华北,745813.51,919744.01,781743.56,2447301.08
西北,267870.77,316212.44,230956.36,815039.57
西南,347692.61,501533.69,453898.21,1303124.51
总计,4865589.87,5734340.95,5469023.47,16068954.29


In [73]:
# 以大区、省份分组，列标签：一级分类，聚合：数量（sum）和金额（mean），显示总计
tb = pd.pivot_table(
    data=df,
    index=['大区', '省份'],
    columns='一级分类',
    values=['数量', '金额'],
    # aggfunc=['sum', 'mean'],
    aggfunc={'数量': 'sum', '金额': 'mean'},
    margins=True,
    margins_name='总计',
    fill_value=0)
# for x, y in tb.columns:
#     print(y+'_'+x)
tb.columns = [y+'_'+x for x, y in tb.columns]
tb = tb.reset_index()
tb.head()

,大区,省份,办公_数量,家具_数量,技术_数量,总计_数量,办公_金额,家具_金额,技术_金额,总计_金额
0,东北,吉林,622,315,361,1298,696.637076,2684.229036,3314.229333,1861.036599
1,东北,辽宁,1476,637,534,2647,807.358976,1810.774774,1828.639067,1257.390262
2,东北,黑龙江,1524,518,476,2518,1084.317112,2917.743636,2885.868889,1853.461006
3,中南,广东,1822,661,609,3092,849.228067,2834.180258,3110.816730,1742.121739
4,中南,广西,441,161,195,797,977.576190,2820.650667,2657.290417,1724.446667


# crosstab交叉表

In [76]:
df = pd.read_excel('./data/demo_03.xlsx', sheet_name='Sheet1')

In [81]:
# index, Excel数据透视表的行
# columns, Excel数据透视表的列
# values, 聚合字段
# rownames, 行索引的别名
# colnames, 列索引的别名
# aggfunc, 聚合函数
# margins, 是否显示合计
# margins_name, 合计的别名
# dropna, 是否删除缺失行（整行缺失）
# normalize, Excel数据透视表的值的显示方式：总计百分比、行汇总百分比、列汇总百分比
# normalize=True, 总计百分比
# normalize='all', 总计百分比
# normalize='index', 行汇总百分比
# normalize='columns', 列汇总百分比
pd.crosstab(
    index=df['大区'],
    columns=df['一级分类'],
    values=df['数量'],
    aggfunc='sum',
    margins=True,
    margins_name='总计',
    normalize='columns')

一级分类,办公,家具,技术,总计
大区,,,,
东北,0.169244,0.173124,0.179403,0.172191
中南,0.261203,0.238252,0.273096,0.258432
华东,0.296295,0.296432,0.285658,0.294160
华北,0.141115,0.141208,0.121303,0.137102
西北,0.045325,0.055117,0.045407,0.047557
西南,0.086818,0.095866,0.095132,0.090558


# category数据类型
- 主要为了提升运算性能

In [83]:
df = pd.read_csv('./data/demo_03_cat.csv')
df.head()

,大区,金额
0,西南,270
1,华北,960
2,东北,701
3,华东,425
4,中南,582


In [84]:
# df.shape

(1048575, 2)

In [85]:
# df.dtypes

大区    object
金额     int64
dtype: object

- 测试非category数据类型的运算时间

In [86]:
%%time
df.groupby('大区')['金额'].sum()

Wall time: 141 ms


大区
东北    96128025
中南    96518713
华东    96023787
华北    96369953
西北    96451240
西南    95335807
Name: 金额, dtype: int64

- 测试category数据类型的运算时间

In [87]:
df['大区'] = df['大区'].astype('category')

In [88]:
df.dtypes

大区    category
金额       int64
dtype: object

In [89]:
%%time
df.groupby('大区')['金额'].sum()

Wall time: 16.9 ms


大区
东北    96128025
中南    96518713
华东    96023787
华北    96369953
西北    96451240
西南    95335807
Name: 金额, dtype: int64